# Python Beyond Basics (Lesson 6)

> Context Manager


## what is a context manager
```
with context-manager:

    context-manager.begin() #　enter()
    body
    context-manager.end() # exit()
```

## Context Protocol

`__enter__(self)`

`__exit__(self, exc_type, exc_val, exc_tb)`

![Context Protocol](https://i.loli.net/2018/01/15/5a5c5d2e57060.png)


In [1]:
# Example 
# LoggingContextManager

class LoggingContextManager:
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        return

In [2]:
with LoggingContextManager() as x:
    print(x)

In [3]:
class LoggingContextManager:
    
    def __enter__(self):
        return 'this is LoggingContext'
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        return
    


In [5]:
with LoggingContextManager() as x:
    print(type(x))
    print(x)

<class 'str'>
this is LoggingContext


In [6]:
class LoggingContextManager:
    
    def __enter__(self):
        print('LoggingContextManager.__enter__()')
        return 'this is LoggingContext'
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('LoggingContextManager.__exit__({},{},{})'.format(
            exc_type, exc_val, exc_tb
        ))
        return
    

In [7]:
with LoggingContextManager() as x:
    print(x)

LoggingContextManager.__enter__()
this is LoggingContext
LoggingContextManager.__exit__(None,None,None)


In [8]:
with LoggingContextManager() as x:
    raise ValueError('Something wrong happened.')
    print(x)

LoggingContextManager.__enter__()
LoggingContextManager.__exit__(<class 'ValueError'>,Something wrong happened.,<traceback object at 0x11286e388>)


ValueError: Something wrong happened.

In [9]:
f = open('a_file', 'w')
with f as g:
    print(f is g)

True


In [10]:
try:
    with LoggingContextManager() as x:
        raise ValueError('Value Error')
except ValueError as e:
    print('*** Value Error detected ***')

LoggingContextManager.__enter__()
LoggingContextManager.__exit__(<class 'ValueError'>,Value Error,<traceback object at 0x112aae0c8>)
*** Value Error detected ***


* If `__exit__()` returns False, the exception is propagated
* `__exit__()` answer the question "should the with-statement swallow exceptions?"
* By default return None, None evaluates False
* `__exit__()` should never expliitly re-raise exceptions
* `__exit__()` should only raise exceptions if it fails itself



## Contextlib

standard library module for working with context managers

contextlib.contextmanager

```
@contextmanager
def my_context_manager():
    # enter
    try:
        yield [value]
        # normal exit
    except:
        # Exception exit
        raise
```

import contextlib
import sys

@contextlib.contextmanager
def logging_context_manager():
    print('logging context manager .__enter__()')
    try:
        yield 'this is LoggingContext'
        print('logging context manager normal exit()')
    except Exception:
        print('logging context manager: exceptional exit', sys.exc_info())
        # 如果要 re-raise Exception 加 raise
        # 不加 相当于  return True
        raise

In [23]:
with logging_context_manager() as x:
    print(x)

logging context manager .__enter__()
this is LoggingContext
logging context manager normal exit()


In [24]:
with logging_context_manager() as x:
    raise ValueError('Value Error happened.')
    print(x)

logging context manager .__enter__()
logging context manager: exceptional exit (<class 'ValueError'>, ValueError('Value Error happened.',), <traceback object at 0x112b231c8>)


ValueError: Value Error happened.

## Multiple Context Manager

```
    with cm1() as a, cm2() as b:
        body
    
    
    # 相当于
    
    with cm1() as a:
        with cm2() as b:
            body
```

In [25]:
@contextlib.contextmanager
def nest_name(name):
    print('enter:', name)
    yield
    print('exit:', name)

In [26]:
with nest_name('outer'), nest_name('inner'):
    print('BODY')

enter: outer
enter: inner
BODY
exit: inner
exit: outer


In [29]:
@contextlib.contextmanager
def nest_name(name):
    print('enter:', name)
    yield name
    print('exit:', name)
    

In [30]:
with nest_name('outer') as n1, nest_name('inner, nested in '+n1):
    print('BODY')

enter: outer
enter: inner, nested in outer
BODY
exit: inner, nested in outer
exit: outer
